In [1]:
import numpy as np
from Source import RS64_Encoder
from Source import RS64_Decoder
from Source.GF64 import GFE

In [2]:
m = 6
t_RS = 8
b_RS = 1
order_alpha=(2**m)-1
length_parity=2*t_RS
n_RS=order_alpha
k_RS=n_RS - length_parity

Z_RS=[]
for i in range(2*t_RS):    
    Z_RS.append((b_RS+i)%order_alpha)
    
g_x = RS64_Encoder.Generation_Polynomial(Z_RS, order_alpha)

codeword_size = 10

In [3]:
total_SER = np.zeros(11)

file_buffer = 'Result/rscode.csv'
model_f = open(file_buffer, 'w')

In [4]:
for i in range(0, 11):
    SER = 0
    print('snr', i)
    for j in range(10):
        msg_RS, codeword_RS = RS64_Encoder.Encoder(order_alpha, k_RS, codeword_size, g_x)
        vector_codeword_RS = RS64_Encoder.trans_vector(codeword_RS, codeword_size, n_RS, m)
        vector_codeword_RS = np.where(vector_codeword_RS > 0, -1, 1)
        
        Eb_No = i
        recived_vector_RS = RS64_Decoder.Receive_Bit(Eb_No, vector_codeword_RS, k_RS, n_RS)
    
        recived_code = RS64_Decoder.Trans_symbol(recived_vector_RS.flatten(), m)
        
        syndrome_RS, syndrome_check = RS64_Decoder.Syndrome_Check(t_RS, recived_code, order_alpha, Z_RS)
        estimated_code = RS64_Decoder.Decoder(syndrome_RS, syndrome_check, recived_code, t_RS, order_alpha, m)
        
        estimated_code = np.reshape(estimated_code,(codeword_size, n_RS))
        
        SER = RS64_Decoder.SER_Calculation(estimated_code, msg_RS, length_parity)
        
    total_SER[i] = SER / (10)
    
    write_buffer = 'Eb_No, %f, ' %(i)
    model_f.write(write_buffer)
    
    write_buffer = 'SER, %.10f\n' % (total_SER[i])
    model_f.write(write_buffer)
    
model_f.close()

snr 0
snr 1
snr 2
snr 3
snr 4
snr 5
snr 6
snr 7
snr 8
snr 9
snr 10
